In [ ]:
#!pip install pypdf
#!pip install langchain
#!pip install gradio
#!pip install docx2txt
#!pip install google-search-results

In [69]:
import os
import re
import json
import boto3
import asyncio
from botocore.config import Config
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.agents import Tool, AgentExecutor, AgentOutputParser
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks import AsyncIteratorCallbackHandler
from typing import Dict
from typing import Any, Dict, List, Optional,Union


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
  
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]
        

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    client_kwargs["aws_access_key_id"] = os.environ.get("AWS_ACCESS_KEY_ID","")
    client_kwargs["aws_secret_access_key"] = os.environ.get("AWS_SECRET_ACCESS_KEY","")
    
    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client



## for aksk bedrock
def get_bedrock_aksk(secret_name='chatbot_bedrock', region_name = "us-west-2"):
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])
    return secret['BEDROCK_ACCESS_KEY'],secret['BEDROCK_SECRET_KEY']

ACCESS_KEY, SECRET_KEY=get_bedrock_aksk()

#role based initial client#######
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "default"
#os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123"  # E.g. "arn:aws:..."
os.environ["AWS_ACCESS_KEY_ID"]=ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"]=SECRET_KEY


#新boto3 sdk只能session方式初始化bedrock
boto3_bedrock = get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

parameters_bedrock = {
    "max_tokens_to_sample": 2048,
    #"temperature": 0.5,
    "temperature": 0,
    #"top_k": 250,
    #"top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

bedrock_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock,model_kwargs=parameters_bedrock)
#####langchain bedrock 不支持异步apredict，无法流式输出##########
#callback = AsyncIteratorCallbackHandler()
#bedrock_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, streaming=True,callbacks=[callback],model_kwargs=parameters_bedrock)
#coro = bedrock_llm.apredict("香格里拉在哪里?\n")
#asyncio.create_task(coro)
#async for token in callback.aiter():
#    print(token)


class BedrockModelWrapper(Bedrock):
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        prompt = "\nHuman: \n" + prompt + "\nAssistant:"   ## Satisfy Bedrock-Claude prompt requirements
        return super()._call(prompt, stop, run_manager, **kwargs)


bedrock_llm_additional = BedrockModelWrapper(model_id="anthropic.claude-v2", 
                                          client=boto3_bedrock, 
                                          model_kwargs=parameters_bedrock)


##use customerized outputparse to fix claude not match 
##langchain's openai ReAct template don't have 
##final answer issue 
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        #print("cur step's llm_output ==="+llm_output)
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output},
                log=llm_output,
            )
            #raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

Create new client
  Using region: us-west-2
  Using profile: default
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


TypeError: a coroutine was expected, got ' 香格里拉位于中国西藏自治区东南部,是传说中的极乐世界。主要有以下几点关于香格里拉的地理位置:\n\n1. 香格里拉位于横断山脉的怒江河谷地带,海拔约3300米,周围被高山环抱。\n\n2. 它属于迪庆藏族自治州管辖,地处云南省西北部。距离丽江市约430公里,离省会昆明市约680公里。\n\n3. 香格里拉县城位于金沙江和白马雪山之间,拥有美丽的自然风光和藏族传统文化。\n\n4. 香格里拉是著名的旅游胜地,有“东方小瑞士”之称。主要景点有松赞林寺、白水台等。\n\n5. 1999年,中国政府在这里建立了香格里拉市,并办起了香格里拉亚洲论坛。\n\n6. 香格里拉在藏语中的意思是“心中的日月”。它来源于英国作家詹姆斯·希尔顿的小说《消失的地平线》,代表理想家园。'

In [56]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.schema import BaseRetriever
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import BaseRetriever, Document
import os




class FullContentRetriever(BaseRetriever):
    doc_path={}
    def _get_content_type(self,doc_path:str):
        fullname=""
        for root, dirs, files in os.walk(doc_path):
            for f in files:
                fullname = os.path.join(root, f)
                ext = os.path.splitext(fullname)[1]
                self.doc_path[fullname]=ext
        print(self.doc_path)        

        
    
    def _get_relevant_documents(
        self,
        query: str,
        *,
        run_manager: CallbackManagerForRetrieverRun,
        **kwargs: Any,
    ) -> List[Document]:
        allDocs =[]
        for key,value in self.doc_path.items():
            #print("key:"+key+" value:"+value)
            if value == ".doc":
                word_loader = Docx2txtLoader(key)
                word_document = word_loader.load()
                allDocs.append(word_document)
            elif value == ".txt":
                txt_loader = TextLoader(key)
                txt_document = txt_loader.load()
                allDocs.append(txt_document)
            elif value == ".pdf":
                pdf_loader = PyPDFLoader(self.doc_path)
                pdf_document = pdf_loader.load()
                allDocs.append(pdf_document)
            else:
                pass
        return allDocs


In [57]:
from langchain.utilities import SerpAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import os


retriever = FullContentRetriever()
retriever._get_content_type("./docs")
retriever_tool = create_retriever_tool(
    retriever,
    "search enterprise documents",
    "useful for when you need to retreve documents regarding the user's question",
)
os.environ["SERPAPI_API_KEY"]="e94267b343a2985d25d7a9a65e1b31a6629a4b4860872c927b33c88674fa89d2"
search = SerpAPIWrapper()
search_tool = Tool(
    name="search website",
    func=search.run,
    description="useful for when you need to answer questions by searching the website",
)
custom_tool_list = [retriever_tool,search_tool]

{'./docs/aws_emr.txt': '.txt', './docs/aws_cleanroom.txt': '.txt', './docs/.ipynb_checkpoints/aws_cleanroom-checkpoint.txt': '.txt'}


## 测试tools

In [64]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain import LLMChain


# Initialize search agent
output_parser = CustomOutputParser()
agent_executor = initialize_agent([retriever_tool], bedrock_llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         agent_kwargs={
                            "output_parser": output_parser
                         },
                         verbose=True)
agent_prompt=agent_executor.agent.llm_chain.prompt
result=agent_executor.run("AWS Clean Rooms怎么收费的?")
result



> Entering new AgentExecutor chain...
 Here is how I would answer the question about AWS Clean Rooms pricing:

Question: AWS Clean Rooms怎么收费的?
Thought: Since this is a question about AWS pricing, I should search AWS documentation for information on Clean Rooms pricing.
Action: search enterprise documents
Action Input: "AWS Clean Rooms pricing" Here is how I would answer the question about AWS Clean Rooms pricing:

Question: AWS Clean Rooms怎么收费的?
Thought: Since this is a question about AWS pricing, I should search AWS documentation for information on Clean Rooms pricing.
Action: search enterprise documents
Action Input: "AWS Clean Rooms pricing"
Observation: [[Document(page_content='Question: Amazon EMR允许处理哪些类型的数据和业务情报工作负载？\nAnswer: 使用Amazon EMR，您可以处理用于分析目的的数据和业务情报工作负载。\n=====\nQuestion: Amazon EMR允许将数据移出/移入到哪些AWS数据存储和数据库中？\nAnswer: Amazon EMR允许您转换大量数据并移出/移入到其它AWS数据存储和数据库中，例如Amazon Simple Storage Service（Amazon S3）和Amazon DynamoDB。\n=====\nQuestion: Amazon EMR的教程在哪里可以找到？\nAnswer: Amaz

'AWS Clean Rooms is charged based on CRPU-hour usage at a rate of $0.656 per CRPU-hour. The cost of a query is calculated by multiplying the query duration in hours (rounded up to the nearest minute) by 32 CRPUs, and then multiplying the CRPU-hours by the per CRPU-hour price. Usage and costs can be viewed in the AWS Billing console.'

In [65]:
response=agent_executor.run("AWS EMR有哪些节点类型?")
print(type(response))



> Entering new AgentExecutor chain...
 Here is my response:

Question: AWS EMR有哪些节点类型?
Thought: To answer this, I should search for AWS EMR documentation that covers the different node types.
Action: search enterprise documents
Action Input: "aws emr node types" Here is my response:

Question: AWS EMR有哪些节点类型?
Thought: To answer this, I should search for AWS EMR documentation that covers the different node types.
Action: search enterprise documents
Action Input: "aws emr node types"
Observation: [[Document(page_content='Question: Amazon EMR允许处理哪些类型的数据和业务情报工作负载？\nAnswer: 使用Amazon EMR，您可以处理用于分析目的的数据和业务情报工作负载。\n=====\nQuestion: Amazon EMR允许将数据移出/移入到哪些AWS数据存储和数据库中？\nAnswer: Amazon EMR允许您转换大量数据并移出/移入到其它AWS数据存储和数据库中，例如Amazon Simple Storage Service（Amazon S3）和Amazon DynamoDB。\n=====\nQuestion: Amazon EMR的教程在哪里可以找到？\nAnswer: Amazon EMR的教程可以在Amazon EMR服务页面中找到，也可以在Amazon EMR管理指南中找到。\n=====\nQuestion: Amazon EMR集群由哪些组件构成？\nAnswer: Amazon EMR集群由集群和节点组成。集群是Amazon Elastic Compute Cloud（Amazon EC2）实

In [ ]:
del agent 
del agent_executor

## 测试agent

In [45]:
from langchain.memory import ConversationBufferMemory
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationBufferWindowMemory(k=2,memory_key="chat_history", input_key='input', output_key="output")
PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""

SUFFIX = """Begin!

Question: {input}
Thought:{agent_scratchpad}"""

customerized_instructions="""
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These are guidance on when to use a tool to solve a task, follow them strictly:
 - first use "search enterprise documents" tool to retreve the document to answer if need
 - then use "search website" tool to search the latest website to answer if need
"""

In [46]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

###step by step contstruct########
#llm_chain = LLMChain(llm=bedrock_llm)
#agent = ZeroShotAgent(llm_chain=llm_chain, tools=custom_tool_list, verbose=True)
#agent_chain = AgentExecutor.from_agent_and_tools(
#    agent=agent, tools=custom_tool_list, verbose=True, memory=memory
#)
#agent_chain.run("最近最火的电影是什么？")
##quick construct#####
agent_executor = initialize_agent(custom_tool_list, bedrock_llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                                  verbose=True,max_iterations=5,
                                  handle_parsing_errors=True,
                                  memory=memory,
                                  agent_kwargs={
                                      "output_parser": output_parser,
                                      #'prefix':PREFIX,
                                      #'suffix':SUFFIX,
                                      'format_instructions':customerized_instructions
                                           }
                                 )
#agent_executor.agent.llm_chain.prompt.template
agent_executor.run("九寨沟明天天气如何?")




> Entering new AgentExecutor chain...
 Question: 九寨沟明天天气如何?

Thought: 要回答九寨沟明天的天气情况,我需要先在企业内部文件中搜索相关信息。

Action: search enterprise documents
Action Input: 九寨沟天气预报
Observation: [[Document(page_content='Question: Amazon EMR允许处理哪些类型的数据和业务情报工作负载？\nAnswer: 使用Amazon EMR，您可以处理用于分析目的的数据和业务情报工作负载。\n=====\nQuestion: Amazon EMR允许将数据移出/移入到哪些AWS数据存储和数据库中？\nAnswer: Amazon EMR允许您转换大量数据并移出/移入到其它AWS数据存储和数据库中，例如Amazon Simple Storage Service（Amazon S3）和Amazon DynamoDB。\n=====\nQuestion: Amazon EMR的教程在哪里可以找到？\nAnswer: Amazon EMR的教程可以在Amazon EMR服务页面中找到，也可以在Amazon EMR管理指南中找到。\n=====\nQuestion: Amazon EMR集群由哪些组件构成？\nAnswer: Amazon EMR集群由集群和节点组成。集群是Amazon Elastic Compute Cloud（Amazon EC2）实例的集合，集群中的每个实例称作节点。\n=====\nQuestion: Amazon EMR中的节点类型有哪些？\nAnswer: Amazon EMR中的节点类型有主节点、核心节点和任务节点。\n=====\nQuestion: 主节点在Amazon EMR集群中的作用是什么？\nAnswer: 主节点管理集群，它通过运行软件组件来协调在其它节点之间分配数据和任务的过程以便进行处理。主节点跟踪任务的状态并监控集群的运行状况。\n=====\nQuestion: 核心节点在Amazon EMR集群中的作用是什么？\nAnswer: 核心节点具有运行任务并在集群上的Hadoop Distributed File System（HDFS）中存

' 对不起,我无法直接回答九寨沟明天的天气情况,因为我没有访问实时天气预报数据的能力。不过,我可以尝试搜索企业文档,看看是否有九寨沟地区的气候概况或天气特征相关的信息。如果没有找到相关信息,我会考虑搜索网站获得更多关于九寨沟天气的资讯。请给我下一步的指导。'

In [48]:
del agent_executor
del memory

NameError: name 'agent_executor' is not defined

## ui demo 测试

In [ ]:
!python ./demo_ui.py

{'./docs/aws_emr.txt': '.txt', './docs/aws_cleanroom.txt': '.txt', './docs/.ipynb_checkpoints/aws_cleanroom-checkpoint.txt': '.txt'}
Create new client
  Using region: us-west-2
  Using profile: default
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/layouts/column.py:55: UserWarning: 'scale' value should be an integer. Using 4.5 will cause issues.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/layouts/column.py:55: UserWarning: 'scale' value should be an integer. Using 5.5 will cause issues.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/blocks.py:902: UserWarning: api_name execute_agent already exists, using execute_agent_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
4.0.2
Running on local URL:  http://127.0.0.1:7860
Running on public UR

In [ ]:
###UI local test####
!python ./demo_table_ui.py